In [ ]:
pip install pymodbus

In [ ]:
import serial
from pymodbus.pdu import ModbusRequest
from pymodbus.client import ModbusSerialClient as ModbusClient
# from pymodbus.client.sync import ModbusSerialClient as ModbusClient
from pymodbus.transaction import ModbusRtuFramer
from pymodbus.constants import Defaults

from time import sleep
import json

# Константы повторного опроса при пропаже соединенния или пакета
Defaults.RetryOnEmpty = True
Defaults.Timeout = 5
Defaults.Retries = 5

# клиент соединеня по UART
client = ModbusClient(method = 'rtu', 
                    port='/dev/ttyUSB0', 
                    baudrate=115200,
                    timeout=1, 
                    bytesize=8, 
                    stopbits=1)

# modbus protocol IDimport serial
from pymodbus.pdu import ModbusRequest
from pymodbus.client import ModbusSerialClient as ModbusClient
# from pymodbus.client.sync import ModbusSerialClient as ModbusClient
from pymodbus.transaction import ModbusRtuFramer
from pymodbus.constants import Defaults

from time import sleep
import json

# Константы повторного опроса при пропаже соединенния или пакета
Defaults.RetryOnEmpty = True
Defaults.Timeout = 5
Defaults.Retries = 5

# клиент соединеня по UART
client = ModbusClient(method = 'rtu', 
                    port='/dev/ttyUSB0', 
                    baudrate=115200,
                    timeout=1, 
                    bytesize=8, 
                    stopbits=1)

# modbus protocol ID
doors_and_film_system_slave_id = 1

# Utils
def mb_connect():
	# првоеряем подключение, пытаемся подключиться 10 раз
	for i in range(10):
		connection = client.connect()
		if(not connection):
			print("Modbus not connected... try to connect again")
			sleep(0.1)
		else:
			# print("Modbus connected succesfully")
			return connection

	return False


# Двери
class DoorModbusManager(object):
	def mb_read_state(self):
		connection = mb_connect()
		if not connection:
			print('Unable to connect door and film module via Modbus and RS-485.')
			return False, None

		state = client.read_holding_registers(0, 13, slave=doors_and_film_system_slave_id) # start_address, count, slave_id
		
		if state.isError():
			print('Unable to read or write Modbus registers. Error:\n %s', state)
			return False, None

		return True, state.registers

	def mb_write_default_params(self):
		connection = mb_connect()
		if not connection:
			print('Unable to connect door and film module via Modbus and RS-485.')
			return False
		
		extra_step_default_data = [12, 6]
		step_res_default_data = [1, 1]
		command_degault_data = 0

		# записываем регистры и проверяем ответ
		# TODO: сделать несколько попыток
		response1 = client.write_registers(3, extra_step_default_data, slave=doors_and_film_system_slave_id)
		if response1.isError():
			print('Unable to write params or there is the connection problem.')
			return False

		response2 = client.write_registers(3, step_res_default_data, slave=doors_and_film_system_slave_id)
		if response2.isError():
			print('Unable to write params or there is the connection problem.')
			return False
			
		response3 = client.write_registers(3, command_degault_data, slave=doors_and_film_system_slave_id)
		if response3.isError():
			print('Unable to write params or there is the connection problem.')
			return False

		return True

	def mb_write_command(self, command):
		connection = mb_connect()
		if not connection:
			print('Unable to connect door and film module via Modbus and RS-485.')
			return False
	
		response = client.write_register(0, int(command), slave=doors_and_film_system_slave_id)  
		if response.isError():
			print('Unable to read or write Modbus registers. Error:\n %s', response)
			return False

		return True

	def mb_write_speed(self, motor, speed):
		connection = mb_connect()
		if not connection:
			print('Unable to connect door and film module via Modbus and RS-485.')
			return False
		
		if(speed < 100 and speed > 50):
			resolution = 1
		elif(speed <= 50 and speed > 25):
			resolution = 2
		elif(speed <= 25 and speed > 12):
			resolution = 4
		elif(speed <= 12 and speed > 6):
			resolution = 8
		elif(speed <= 6 and speed > 3):
			resolution = 16
		else:
			resolution = 32


		if (motor == 1):
			response = client.write_register(9, int(resolution), slave=doors_and_film_system_slave_id)  
			if response.isError():
				print('Unable to read or write Modbus registers. Error:\n %s', response)
				return False
		else:
			response = client.write_register(10, int(resolution), slave=doors_and_film_system_slave_id)  
			if response.isError():
				print('Unable to read or write Modbus registers. Error:\n %s', response)
				return False

		return True

doors_mb = DoorModbusManager()

# high level state for REST API
def state_high_level():
    is_mb_connect, data = doors_mb.mb_read_state()
    
    if not is_mb_connect:
        print("Not connected")
        error = "error"
        state_upper = "error"
        state_lower = "error"
        return error, state_upper, state_lower 

    error = "not error"
    state_upper = ""
    state_lower = ""

    if(not data[11] == 0):
        error = "error"
        state_upper = "error"
        state_lower = "error"
    elif(data[0] == 0 and data[11] == 0):
        state_upper = "door is unhold"
        state_lower = "door is unhold"
    elif(data[0] == 1 and data[11] == 0):
        if(data[6] == data[3]):
            state_upper = "door closed without errors"
        else:
            state_upper = "door is closing"
        if(data[8] == data[4]):
            state_lower = "door closed without errors"
        else:
            state_lower = "door is closing"
    elif(data[0] == 2 and data[11] == 0):
        if(data[5] == data[1]):
            state_upper = "door open without errors"
        else:
            state_upper = "door is opening"
        if(data[7] == data[2]):
            state_lower = "door open without errors"
        else:
            state_lower = "door is opening"
    else:
        print("some logic error occured")
        state_upper = "error"
        state_lower = "error"
    
    return error, state_upper, state_lower

# для тестирования 
def open_test():
    # print("TEST")
    error, upper_state, lower_state = state_high_level()
    if error == "error":
        print("some error")
        return
        # TODO: print error num
    
    # open door and wait
    result = doors_mb.mb_write_command(2)
    print("OPENING...")
    sleep(0.2)
    while (upper_state != "door open without errors" or lower_state != "door open without errors"):
        error, upper_state, lower_state = state_high_level()
        sleep(0.5)
    print("OPEN")
    return 

def close_test():
    error, upper_state, lower_state = state_high_level()
    if error == "error":
        print("some error")
        return
        # TODO: print error num
    
    # close door and wait
    result = doors_mb.mb_write_command(1)
    print("Closing...")
    sleep(0.2)
    while (upper_state != 'door closed without errors' or lower_state != 'door closed without errors'):
        error, upper_state, lower_state = state_high_level()
        sleep(0.5)
    print("CLOSED")
    return

In [2]:
mb_connect()

True

### Сброс ошибки

In [ ]:
client.write_register(11, 0, slave=doors_and_film_system_slave_id)

### Настройка скорости работы дверей

In [ ]:
client.write_register(12, 1000 , slave=doors_and_film_system_slave_id)

# Состояние системы

In [5]:
is_connect, state = doors_mb.mb_read_state()

if is_connect:
    result = json.dumps(state, indent=4, sort_keys=True)
    print(result)

[
    0,
    1000,
    560,
    115,
    7,
    0,
    0,
    0,
    0,
    1,
    16,
    0,
    200
]


In [8]:
state = client.read_holding_registers(0, 13, slave=doors_and_film_system_slave_id)
# state

14:26:03 WARNING serial:261 Cleanup recv buffer before send: 0x0 0xff


In [3]:
state_high_level()

15:29:56 WARNING serial:261 Cleanup recv buffer before send: 0x0 0xff


('not error', 'door is unhold', 'door is unhold')

# Открыть дверь

In [58]:
result = doors_mb.mb_write_command(2)

13:19:04 WARNING serial:261 Cleanup recv buffer before send: 0xff


# Закрыть дверь

In [16]:
result = doors_mb.mb_write_command(1)

14:35:48 WARNING serial:261 Cleanup recv buffer before send: 0xff


# Отпустить дверь

In [76]:
result = doors_mb.mb_write_command(0)

13:35:23 WARNING serial:261 Cleanup recv buffer before send: 0xff


# Открыть и закрыть дверь N раз

In [ ]:
num = 4
for i in range(num):
    print("========= TEST " + str(i) + " =========")
    open_test()
    sleep(3)
    close_test()
    sleep(3)

In [ ]:
open_test()

In [ ]:
close_test()

# Протяжка пленки

In [ ]:
state = client.read_holding_registers(100, 11, slave=doors_and_film_system_slave_id) # start_address, count, slave_id
state.registers

### erase error

In [ ]:
client.write_register(100, 0 , slave=doors_and_film_system_slave_id)
client.write_register(101, 0 , slave=doors_and_film_system_slave_id)

### film broach

In [ ]:
client.write_register(102, 200 , slave=doors_and_film_system_slave_id)

# logging

In [50]:
import logging
import os
from logging.handlers import RotatingFileHandler

logger_doors = logging.getLogger('doors_test')
fh = logging.FileHandler('doors_test.log')
# fh.setLevel(logging.INFO)
formatter = logging.Formatter('%(asctime)s - %(name)s - %(levelname)s - %(message)s')
fh.setFormatter(formatter)
logger_doors.addHandler(fh)

In [52]:

logger_doors.info('lol')
logger_doors.error('lol')

13:15:26 ERROR 2457548588:2 lol
